In [1]:

import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

/Users/mas/falcon/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True

# Transaction Classification based on Merchant

In [2]:
url = "https://api.ai71.ai/v1/chat/completions"
AI71_TOKEN = os.getenv('AI71_TOKEN')
role_content = "You will be provided with a company name, and your task is to classify it to fitness, food, healthcare, clothing, transportation, fun, utilities, personal care, shopping, miscellaneous"
user_content = "CLASSPASS* MONTHLY CLASSPASSCOM"

payload = json.dumps({
  "model": "tiiuae/falcon-180b-chat",
  "messages": [
    {
      "role": "system",
      "content": role_content
    },
    {
      "role": "user",
      "content": user_content
    }
  ]
})
headers = {
  'Content-Type': 'application/json',
  'Authorization': f'Bearer {AI71_TOKEN}'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"id":"RequestType.CHAT_COMPLETION-tiiuae/falcon-180b-chat-2ffbf6db-b047-4dab-b09c-bfd60696f919-api71-api-2ffbf6db-b047-4dab-b09c-bfd60696f919-7a4056e1-e94f-4660-af57-f7ccfa99691b","created":1722075918,"model":"tiiuae/falcon-180b-chat","usage":{"completion_tokens":2,"prompt_tokens":60,"total_tokens":62},"choices":[{"finish_reason":"stop","index":0,"message":{"content":" Fitness","role":"assistant"}}],"object":"chat.completion"}


In [3]:
try:
    response_json = response.json()
    choices = response_json.get('choices', [])
    if choices:
        message_content = choices[0].get('message', {}).get('content', '')
        print(f'Classification Result: {message_content.strip()}')
    else:
        print("No choices found in the response.")
        
except ValueError:
    print("Response is not in JSON format:")
    print(response.text)

Classification Result: Fitness


### Classify Bank Transactions

In [4]:
df = pd.read_csv('../data/sample_transactions.csv')
df.head()

,Date,Description,Merchant,Amount
0,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,8.70
1,07 Jul 2024,CARD NO.4439********1246 AL AQSA GAR AND BLANK...,AL AQSA GAR AND BLANKE,85.00
2,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.55
3,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.39
4,05 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 0835...,Talabat,139.63


In [5]:
import os
import json
import requests

def classify_company(user_content):
    url = "https://api.ai71.ai/v1/chat/completions"
    AI71_TOKEN = os.getenv('AI71_TOKEN')

    categories = [
    'fitness',
    'groceries',
    'restaurants and cafes',
    'healthcare', 
    'clothing',
    'jewelry',
    'transportation',
    'phone and internet', 
    'miscellaneous',
    'others',
    'e-commerce',
    'food delivery'
    ]

    categories_str = ', '.join(categories)

    role_content = (
        f"You will be provided with a company name, and your task is to classify it to one of the following "
        f"{categories_str}"
    )
    
    payload = json.dumps({
        "model": "tiiuae/falcon-180b-chat",
        "messages": [
            {
                "role": "system",
                "content": role_content
            },
            {
                "role": "user",
                "content": user_content
            }
        ]
    })
    
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {AI71_TOKEN}'
    }
    
    response = requests.post(url, headers=headers, data=payload)

    try:
        response_json = response.json()
        choices = response_json.get('choices', [])
        if choices:
            result = choices[0].get('message', {}).get('content', '').strip()
            return result
        else:
            return None
            
    except ValueError:
        return None

 
classify_company("CLASSPASS* MONTHLY CLASSPASSCOM")

'Fitness'

In [8]:
df['Category_freetext'] = df['Merchant'].apply(classify_company)

In [9]:
df

,Date,Description,Merchant,Amount,Category,Category_freetext
0,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,8.70,groceries,groceries
1,07 Jul 2024,CARD NO.4439********1246 AL AQSA GAR AND BLANK...,AL AQSA GAR AND BLANKE,85.00,Clothing,Clothing
2,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.55,groceries,groceries
3,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.39,groceries,groceries
4,05 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 0835...,Talabat,139.63,Food delivery,Food delivery
5,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,50.98,groceries,groceries
6,04 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,17.25,groceries,groceries
7,03 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 6897...,Talabat,66.04,Food delivery,Food delivery
8,03 Jul 2024,CARD NO.4439********1246 Amazon.ae Dubai:AE 50...,Amazon.ae,175.25,E-commerce,E-commerce
9,02 Jul 2024,CARD NO.4439********1246 UNION COOP DUBAI:AE 2...,UNION COOP,5.50,groceries,groceries


### Standarize the Category

In [10]:
categories = ['fitness', 'groceries', 'restaurants and cafes', 'healthcare', 'clothing', 'jewelry', 
         'transportation', 'phone and internet', 'miscellaneous', 'others', 'e-commerce', 'food delivery']

def find_first_match(text, categories ):
    for category in categories :
        if category in text.lower():
            return category
    return None

df['dCategory'] = df['Category_freetext'].apply(lambda x: find_first_match(x, categories))
df


,Date,Description,Merchant,Amount,Category,Category_freetext,dCategory
0,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,8.70,groceries,groceries,groceries
1,07 Jul 2024,CARD NO.4439********1246 AL AQSA GAR AND BLANK...,AL AQSA GAR AND BLANKE,85.00,Clothing,Clothing,clothing
2,07 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.55,groceries,groceries,groceries
3,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,31.39,groceries,groceries,groceries
4,05 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 0835...,Talabat,139.63,Food delivery,Food delivery,food delivery
5,05 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,50.98,groceries,groceries,groceries
6,04 Jul 2024,CARD NO.4439********1246 NESTO HYPERMARKET LLC...,NESTO HYPERMARKET LLC BRA,17.25,groceries,groceries,groceries
7,03 Jul 2024,CARD NO.4439********1246 Talabat DUBAI:AE 6897...,Talabat,66.04,Food delivery,Food delivery,food delivery
8,03 Jul 2024,CARD NO.4439********1246 Amazon.ae Dubai:AE 50...,Amazon.ae,175.25,E-commerce,E-commerce,e-commerce
9,02 Jul 2024,CARD NO.4439********1246 UNION COOP DUBAI:AE 2...,UNION COOP,5.50,groceries,groceries,groceries
